Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = '../../Data/Tutorial/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [7]:
batch_size = 128
hidden_layer_size = 250

graph = tf.Graph()

with graph.as_default() :
    
    # Layer 1
    weights_layer_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases_layer_1 = tf.Variable(tf.zeros(hidden_layer_size))
    
    # Layer 2
    weights_layer_2 = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_layer_2 = tf.Variable(tf.zeros(num_labels))
    
    def model(tf_train_dataset) : 
        out_layer_1 = tf.matmul(tf_train_dataset, weights_layer_1) + biases_layer_1
        #dense_layer = tf.layers.dense(inputs=out_layer_1, units=hidden_layer_size, activation=tf.nn.relu)
        return tf.matmul(tf.nn.relu(out_layer_1), weights_layer_2) + biases_layer_2
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labelset = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    logits = model(tf_train_dataset)
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labelset))
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(loss)
    train_predictions = tf.nn.softmax(logits=logits)
    valid_predictions = tf.nn.softmax(logits=model(tf_valid_dataset))
    test_predictions = tf.nn.softmax(logits=model(tf_test_dataset))

In [9]:
num_steps = 30001

with tf.Session(graph=graph) as sess :
    tf.global_variables_initializer().run()
    print ("Initialized...")
    
    for step in range(num_steps) :
        
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:offset + batch_size, : ]
        batch_labels = train_labels[offset:offset + batch_size, : ]
        
        feed_dict = {
            tf_train_dataset : batch_data
            , tf_train_labelset : batch_labels
        }
        
        _, l, predictions = sess.run([optimizer, loss, train_predictions], feed_dict)
        
        
        
        if (step % 500 == 0) :
            print ("Loss after mini batch step %d is : %f" % (step, l))
            print ("Mini batch accuracy : %f" % accuracy(predictions, batch_labels))
            print ("Validation accuracy : %f" % accuracy(valid_predictions.eval(), valid_labels))
    print ("Test accuracy : %f" % accuracy(test_predictions.eval(), test_labels))

Initialized...
Loss after mini batch step 0 is : 179.462479
Mini batch accuracy : 8.593750
Validation accuracy : 27.010000
Loss after mini batch step 500 is : 7.316789
Mini batch accuracy : 71.093750
Validation accuracy : 73.760000
Loss after mini batch step 1000 is : 1.991946
Mini batch accuracy : 71.093750
Validation accuracy : 75.640000
Loss after mini batch step 1500 is : 1.319514
Mini batch accuracy : 78.906250
Validation accuracy : 78.740000
Loss after mini batch step 2000 is : 4.111946
Mini batch accuracy : 80.468750
Validation accuracy : 77.350000
Loss after mini batch step 2500 is : 2.214208
Mini batch accuracy : 78.125000
Validation accuracy : 79.450000
Loss after mini batch step 3000 is : 0.864661
Mini batch accuracy : 82.031250
Validation accuracy : 80.750000
Loss after mini batch step 3500 is : 0.968468
Mini batch accuracy : 85.156250
Validation accuracy : 78.130000
Loss after mini batch step 4000 is : 1.433520
Mini batch accuracy : 81.250000
Validation accuracy : 79.03000

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
